In [1]:
import pandas as pd
import numpy as np

import copy
import itertools
import os
import re

from voting_lib import \
    ConvertResponsesToNumericVotes, \
    RepairVote, \
    PerformRankedChoiceVoting, \
    RemovePolicy, \
    RunBordaCountVote, \
    POLICY_NAMES, \
    PrintPolicyVec, \
    RunMultiRoundVoting

# Erroneous input (two policies ranked the same) is resolved randomly.
# Set the seed to prevent flaky voting.
np.random.seed(42)

# Loading and preprocessing results

In [2]:
print('The policies in order of votes are:')
print(POLICY_NAMES)

The policies in order of votes are:
['Policy A', 'Policy B', 'Policy C', 'Policy D']


In [3]:
working_dir = '/home/rgiordan/Documents/git_repos/Presentations/CCC_covid_vote'

In [4]:
#date_string = '11-08_17_02_26'
# raw_responses = pd.read_csv(os.path.join(working_dir, 
#     'Children\'s_Community_Center_Out2022-' + date_string + '.csv'))

# In final_votes.csv I have deleted a duplicate and test vote, and notated which
# yard the parents are from.
raw_responses = pd.read_csv(os.path.join(working_dir, 'final_votes.csv'))

In [5]:
# Compile names
first_names = raw_responses[raw_responses.columns[1]].fillna('').to_numpy()
last_names = raw_responses[raw_responses.columns[2]].fillna('').to_numpy()
raw_full_names = np.array([ x + ' ' + y for x, y in zip(first_names, last_names) ])

In [6]:
# staff_names = [
#     'Allie Pollak',
#     'Edna Tow',
#     'Lara Gabato ',
#     'Allyssa Adair',
#     'Kathy Chew',
#     'Ryan Farrell',
#     'Malcolm Waugh',
#     'Lilian Zakki',
#     'laura McCaul',
#     'Devaki  Nirula ',
#     'Encian  Pastel '
# ]

# Eliminate some rows by full name.
# Test Test was, as it says, a test.
# Stefani apparently voted twice, so remove this one.
bad_names = ['Test Test', 'Stefani Madril']
keep_rows = [ name not in bad_names for name in raw_full_names ] 

# Note that the board decided to keep the one parent who didn't write their
# name rather than go through the trouble of trying to find them.
responses = raw_responses[keep_rows]
full_names = raw_full_names[keep_rows]
print(f'There were {len(full_names)} voters:\n\n', '\n'.join(full_names))


There were 52 voters:

 Ayako Hiwasa
Arielle Ryden
Jaffer Abbasi
Joe Graves
Chris Madril 
Matt Vander Sluis
Emily Schlessinger
Devaki  Nirula 
laura McCaul
Dora Zhang
Allie Pollak
Lilian Zakki
Jennifer  Oppeau
Malcolm Waugh
Juliana Monin
Harriet  Blackburn 
Edna Tow
Natanya Marks
Frances Schaeffer
Jennifer Nakata
Stevie Schwartz
ADELE SHEA
Roya Clune
Zach Alexander
Meghana Gadgil
Lara Gabato 
Sarah Chenoweth 
Theresa Keating
Jose Aranda
jessica thomas
Nathaniel Wolf
Lorian Schaeffer
Torrey Mansur
Jamie Greenwood
Allyssa Adair
Kathy Chew
Alli Beltz
ilyse magy
Miriam Wolodarski Lundberg
Mónica Henestroza
Shaye Mckenney 
Bren Darrow
Auddi Leos
Ian Wulfson
Jill Fox
Dorothy  Thai
 
Ryan Farrell
Clare Armbruster
Encian  Pastel 
Xanh  Văn Ginzburg 
Stephanie and Pete Guinosso


In [7]:
# # Get the indices of staff responses for separate analysis.
# staff_inds = []
# for staff_name in staff_names:
#     inds = np.argwhere(full_names == staff_name).flatten()
#     if len(inds) == 0:
#         print(f'{staff_name} not found')
#         assert(False)
#     staff_inds.append(inds[0])

# assert(np.all(full_names[staff_inds] == staff_names))

In [8]:
original_votes = ConvertResponsesToNumericVotes(responses)

In [9]:
# Fix malformed votes.
votes = copy.copy(original_votes)
for voter in range(votes.shape[1]):
    votes[:, voter] = RepairVote(votes[:, voter])
    
# Report on any votes that were modified.
for voter in range(votes.shape[1]):
    if np.any(votes[:, voter] != original_votes[:, voter]):
        print('\n==============================================')
        print(f'\nThe vote for voter {voter} was repaired:')
        PrintPolicyVec(original_votes[:, voter]) 
        print('...became...')
        PrintPolicyVec(votes[:, voter]) 
        print('\nThe process was as follows:')
        RepairVote(original_votes[:, voter], verbose=True)



The vote for voter 17 was repaired:
Policy A  Policy B  Policy C  Policy D  
3         2         2         1         
...became...
Policy A  Policy B  Policy C  Policy D  
4         3         2         1         

The process was as follows:
Original vote: [3 2 2 1]
Rank 1.  Current vote: [3 2 2 1]
Rank 2.  Current vote: [3 2 2 1]
Rank 2 duplicated, randomly splitting indices [1 2]
Rank 3.  Current vote: [4 3 2 1]
Rank 4.  Current vote: [4 3 2 1]
Final vote:
Policy A  Policy B  Policy C  Policy D  
4         3         2         1         


The vote for voter 48 was repaired:
Policy A  Policy B  Policy C  Policy D  
3         4         3         1         
...became...
Policy A  Policy B  Policy C  Policy D  
2         4         3         1         

The process was as follows:
Original vote: [3 4 3 1]
Rank 1.  Current vote: [3 4 3 1]
Rank 2.  Current vote: [3 4 3 1]
Rank 2 missing, decrementing other votes
Rank 2.  Current vote: [2 3 2 1]
Rank 2 duplicated, randomly splitting indice

# Run the vote

In [10]:
print('Results for the whole school:')

RunMultiRoundVoting(votes)

Results for the whole school:

Round 0:

Checking for a majority.

Number of first-place votes:
Policy A  Policy B  Policy C  Policy D  
4         16        8         24        

The proportion of first votes was
Policy A  Policy B  Policy C  Policy D  
0.0769    0.3077    0.1538    0.4615    

There was no majority.
Removing Policy A

Checking for a majority.
This many voters did not vote this round: 1

Number of first-place votes:
Policy A  Policy B  Policy C  Policy D  
0         19        8         24        

The proportion of first votes was
Policy A  Policy B  Policy C  Policy D  
0.0       0.3725    0.1569    0.4706    

There was no majority.
Removing Policy C

Checking for a majority.
This many voters did not vote this round: 2

Number of first-place votes:
Policy A  Policy B  Policy C  Policy D  
0         23        0         27        

The proportion of first votes was
Policy A  Policy B  Policy C  Policy D  
0.0       0.46      0.0       0.54      
The winner was Policy D

In [20]:
print('We agreed to instant runoff voting, not Borda count.')

print('However, in case you are curious, the Borda count results is as follows (lower is better):')
PrintPolicyVec(RunBordaCountVote(votes))

We agreed to instant runoff voting, not Borda count.
However, in case you are curious, the Borda count results is as follows (lower is better):
Policy A  Policy B  Policy C  Policy D  
152       113       110       114       


## Staff, FY, and BY votes

In [12]:
yard = responses[responses.columns[10]].fillna('').to_numpy()

In [22]:
# Run the auction separately for the staff who voted.  This is only informative,
# not binding.

staff_names = full_names[yard == 'Staff']
print('Results only with staff ', ', '.join(staff_names))
RunMultiRoundVoting(votes[:, yard == 'Staff'], verbose=False)

print('\nBorda count:')
PrintPolicyVec(RunBordaCountVote(votes[:, yard == 'Staff']))

Results only with staff  Devaki  Nirula , laura McCaul, Allie Pollak, Lilian Zakki, Malcolm Waugh, Edna Tow, Lara Gabato , Allyssa Adair, Kathy Chew, Ryan Farrell, Encian  Pastel 

Round 0:
*** The winner of this round is ['Policy B']

Round 1:
*** The winner of this round is ['Policy A']

Round 2:
*** The winner of this round is ['Policy C']

Round 3:
*** The winner of this round is ['Policy D']
The winners in order were: 
['Policy B', 'Policy A', 'Policy C', 'Policy D']

Borda count:
Policy A  Policy B  Policy C  Policy D  
26        17        24        34        


In [24]:
# Run the auction separately for the FY and BY.  This is only informative, not binding.

fy_names = full_names[yard == 'FY']
print('Results only with FY families:\n', ', '.join(fy_names))
RunMultiRoundVoting(votes[:, yard == 'FY'], verbose=False)

print('\nBorda count:')
PrintPolicyVec(RunBordaCountVote(votes[:, yard == 'FY']))

Results only with FY families:
 Arielle Ryden, Joe Graves, Chris Madril , Matt Vander Sluis, Juliana Monin, Harriet  Blackburn , Jennifer Nakata, ADELE SHEA, Roya Clune, Theresa Keating, Jose Aranda, jessica thomas, Jamie Greenwood, Bren Darrow, Auddi Leos, Ian Wulfson, Jill Fox, Dorothy  Thai, Clare Armbruster, Xanh  Văn Ginzburg , Stephanie and Pete Guinosso

Round 0:
*** The winner of this round is ['Policy D']

Round 1:
*** The winner of this round is ['Policy C']

Round 2:
*** The winner of this round is ['Policy B']

Round 3:
*** The winner of this round is ['Policy A']
The winners in order were: 
['Policy D', 'Policy C', 'Policy B', 'Policy A']

Borda count:
Policy A  Policy B  Policy C  Policy D  
66        54        44        35        


In [23]:
by_names = full_names[yard == 'BY']
print('Results only with FY families:\n', ', '.join(by_names))
RunMultiRoundVoting(votes[:, yard == 'BY'], verbose=False)

print('\nBorda count:')
PrintPolicyVec(RunBordaCountVote(votes[:, yard == 'BY']))

Results only with FY families:
 Ayako Hiwasa, Jaffer Abbasi, Emily Schlessinger, Dora Zhang, Natanya Marks, Frances Schaeffer, Stevie Schwartz, Zach Alexander, Meghana Gadgil, Sarah Chenoweth , Nathaniel Wolf, Lorian Schaeffer, Torrey Mansur, Alli Beltz, Miriam Wolodarski Lundberg, Mónica Henestroza, Shaye Mckenney 

Round 0:
*** The winner of this round is ['Policy B']

Round 1:
*** The winner of this round is ['Policy D']

Round 2:
*** The winner of this round is ['Policy C']

Round 3:
*** The winner of this round is ['Policy A']
The winners in order were: 
['Policy B', 'Policy D', 'Policy C', 'Policy A']

Borda count:
Policy A  Policy B  Policy C  Policy D  
49        34        37        39        


# Count "unacceptable policy" votes and display comments

In [15]:
# Compile which polcies are indicated as unacceptable.
#print(responses.columns[7])
unacceptables = responses[responses.columns[7]].fillna('').to_numpy()

unacceptable_inds = []
for policy in [ 'A', 'B', 'C', 'D' ]:
    unacceptable_bools = [ re.match(f'Policy {policy}', v) is not None for v in unacceptables ]
    unacceptable_inds.append(np.argwhere(unacceptable_bools).flatten())

print('Count of unacceptables: ')
PrintPolicyVec([ len(v) for v in unacceptable_inds ])



Count of unacceptables: 
Policy A  Policy B  Policy C  Policy D  
7         2         0         6         


In [16]:
print(responses.columns[8])
unacceptable_text = responses[responses.columns[8]].fillna('').to_numpy()
for policy_ind in range(4):
    print(f'\n======================================================\n' + 
          f'Responses for {POLICY_NAMES[policy_ind]} unacceptable:')
    for voter_ind in unacceptable_inds[policy_ind]:
        print(f'\n > {full_names[voter_ind]} (index {voter_ind})')
        print(unacceptable_text[voter_ind])

print(f'\n======================================================\n' + 
f'Text entered with no policy marked unacceptable')

# Print text for folks who did not indicate that anything was unacceptable
for voter_ind in np.setdiff1d(np.arange(votes.shape[1]), np.hstack(unacceptable_inds)):
    if unacceptable_text[voter_ind] != '':
        print(f'\n > {full_names[voter_ind]} (index {voter_ind})')
        print(unacceptable_text[voter_ind])

If you wish to elaborate on why any policy or policies are unacceptable to you and your family, please do so here.

Responses for Policy A unacceptable:

 > Arielle Ryden (index 1)
See below, but omitting an option for the transmission map and mandatory masks during a surge feels irresponsible to our family. 

 > Juliana Monin (index 14)
We haven't been presented with any evidence that masking outdoors is meaningfully safer than not masking. At this point we believe the social costs for children (and adults) far outweigh any reduction in covid transmission. 

 > Zach Alexander (index 23)


 > Jose Aranda (index 28)
Policy B is unacceptable for us because we're in favor or removing masks outdoors (and indoors), and in practice opting in to test for most of the year is non-sense for us.

Policy A is unacceptable because our understanding is that testing is as effective, if not more effective than masks in preventing exposures to COVID-19. Removing the option to test is like deciding to g

In [17]:
# Display other text
print(responses.columns[9])

comment_text = responses[responses.columns[9]].fillna('').to_numpy()

for voter_ind in range(votes.shape[1]):
    if comment_text[voter_ind] != '':
        print(f'\n > {full_names[voter_ind]} (index {voter_ind})')
        print(comment_text[voter_ind])

If you have any other concerns or feedback on the process or proposals, please share them here.

 > Arielle Ryden (index 1)
Our family wants to voice our deep sadness and regret that in spite of a well thought out, appropriately voted on,
and appropriately processed Covid policy that was put into place last year with community engagement, scientific reasoning and support and vote from the board, that approximately 10 out of 50 families changed their minds and/or feel differently now, and have put the board in a predicament to spend tons of time/energy/resources reworking this policy — this is truly frustrating. As an incoming family this year, this well thought out policy was one of the main reasons we chose CCC along of course with the community aspect and wonderful teachers. As a family who is confident in the current policy and feels it is a responsible and scientifically grounded approach, we are deeply concerned that this uprising has brought dissonance among the community and has